In [1]:
import os
import traceback
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema import HumanMessage, SystemMessage
from google import genai
from google.genai import types

In [4]:
class OnsenAgent:
    def __init__(self, model_name:str="gemini-2.5-flash"):
        self.modoel_name = model_name

        api_key = os.getenv("GOOGLE_API_KEY")
        if not api_key:
            raise ValueError("GOOGLE_API_KEY environment variable not set.")
        # Only run this block for Gemini Developer API
        self.client = genai.Client(api_key=api_key)

        self.llm = ChatGoogleGenerativeAI(model=model_name, google_api_key=api_key)
        

    def serach_web(self, user_question:str):
        # Google Search Toolの定義
        # https://ai.google.dev/gemini-api/docs/migrate?hl=ja
        # https://github.com/googleapis/python-genai
        response = self.client.models.generate_content(
            model=self.modoel_name,
            contents=user_question,
            config=types.GenerateContentConfig(
                tools=[
                    types.Tool(
                        google_search=types.GoogleSearch()
                    )
                ],
                response_modalities=["TEXT"],
            )
        )
        return response.candidates[0].content.parts[0].text

    def post_process(self, context:str, user_question:str):
        final_response = self.llm.invoke([
            SystemMessage(content="あなたは東京都内の天然温泉銭湯施設に関するAIエージェントです。以下の検索結果とユーザーの質問に基づいて、正確で役立つ回答を生成してください。"),
            HumanMessage(content=f"検索結果:\n{context}\n\nユーザーの質問: {user_question}")
        ]).content
        return final_response

    def explore(self, user_question: str):
        try:
            context = self.serach_web(user_question)
            return self.post_process(context, user_question)
        except Exception as e:
            error_msg = traceback.format_exc()
            msg = f"エージェントの実行中にエラーが発生しました: {error_msg}"
            raise RuntimeError(msg)

In [5]:
# 環境変数の設定例 (実際にはシェルで設定)
# os.environ["GOOGLE_API_KEY"] = "YOUR_GEMINI_API_KEY"
# os.environ["GOOGLE_CSE_ID"] = "YOUR_GOOGLE_CSE_ID"

agent = OnsenAgent()
print("東京都内天然温泉銭湯施設AIエージェントへようこそ！")
print("質問を入力してください（終了するには 'exit' と入力）。")

# while True:
question = "品川区にある東京都内の天然温泉付き銭湯を探して"
response = agent.explore(question)
print(f"エージェント: {response}")

東京都内天然温泉銭湯施設AIエージェントへようこそ！
質問を入力してください（終了するには 'exit' と入力）。
エージェント: 品川区には、天然温泉を楽しめる銭湯がいくつかあります。

現在営業中または天然温泉であることを明記している主な銭湯は以下の通りです。

*   **武蔵小山温泉 清水湯**: 武蔵小山駅から徒歩5分。黄金の湯と黒湯の2種類の天然温泉が楽しめます。特に黒湯は、きめ細かく澄んだ濃い琥珀色の上質なモール泉として知られています。岩盤浴も人気です。
*   **中延温泉 松の湯**: 中延駅から徒歩2分。ジェットバス、露天風呂、薬湯、水風呂を含む全てのお湯が天然温泉です。本格的なロッキーサウナや、ヒノキの高濃度炭酸露天風呂も楽しめます。
*   **北品川温泉 天神湯**: シャープでモダンなインテリアが特徴の天然温泉付き銭湯で、ミネラル豊富な黒湯の天然温泉が楽しめます。
*   **戸越銀座温泉**: 戸越銀座界隈にある天然温泉を使用した銭湯で、「美人の湯」や「美肌の湯」として知られています。
*   **西品川温泉 宮城湯**: メタケイ酸を含む微黄褐色澄明無味無臭（無色透明に近い）の天然温泉です。

訪問前に、各銭湯の公式サイトやSNSで最新の営業状況をご確認いただくことをお勧めします。
